## UDF ->USER Defined function

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("window").getOrCreate()

sales_data = [
    {"sale_id": 1, "region": "North", "salesperson": "Arun",   "amount": 5000},
    {"sale_id": 2, "region": "South", "salesperson": "Meena",  "amount": 7000},
    {"sale_id": 3, "region": "East",  "salesperson": "Sachin", "amount": 6500},
    {"sale_id": 4, "region": "North", "salesperson": "Arun",   "amount": 8000},
    {"sale_id": 5, "region": "West",  "salesperson": "Priya",  "amount": 4000},
    {"sale_id": 6, "region": "South", "salesperson": "Meena",  "amount": 9000},
    {"sale_id": 7, "region": "East",  "salesperson": "Sachin", "amount": 7500},
    {"sale_id": 8, "region": "West",  "salesperson": "Priya",  "amount": 6000},
    {"sale_id": 9, "region": "North", "salesperson": "Kiran",  "amount": 3000},
    {"sale_id": 10,"region": "South", "salesperson": "Deepa",  "amount": 5500}
]

df1 = spark.createDataFrame(data=sales_data)

In [ ]:
df1.show()

### step 1

In [ ]:
def my_func(x,y):
    if x>5000 and y in ("North","East"):
        return x//100
    else:
        return x//50

### Step 2

In [ ]:
my_udf = F.udf(my_func)

In [ ]:
df1.withColumn("Bonus",my_udf(F.col("amount"),F.col("region"))).show()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("window").getOrCreate()

sales_data = [
    {"sale_id": 1, "region": "North", "salesperson": "Arun",   "amount": 5000},
    {"sale_id": 2, "region": "South", "salesperson": "Meena",  "amount": 7000},
    {"sale_id": 3, "region": "East",  "salesperson": "Sachin", "amount": 6500},
    {"sale_id": 4, "region": "North", "salesperson": "Arun",   "amount": 8000},
    {"sale_id": 5, "region": "West",  "salesperson": "Priya",  "amount": 4000},
    {"sale_id": 6, "region": "South", "salesperson": "Meena",  "amount": 9000},
    {"sale_id": 7, "region": "East",  "salesperson": "Sachin", "amount": 7500},
    {"sale_id": 8, "region": "West",  "salesperson": "Priya",  "amount": 6000},
    {"sale_id": 9, "region": "North", "salesperson": "Kiran",  "amount": 3000},
    {"sale_id": 10,"region": "South", "salesperson": "Deepa",  "amount": 5500}
]

df = spark.createDataFrame(data=sales_data)

def add_bouns_based_region(x,y):
    if x in ("North","South"):
        return y*0.1
    else:
        return y*0.05

my_fun = F.udf(add_bouns_based_region)

df.withColumn("bonus",my_fun(F.col("region"),F.col("amount")))\
  .withColumn("Bonus %",F.when(((F.col("region")=="North") | (F.col("region")=="South")),"10% bonus")\
    .otherwise("5% bonus")).show()

#or 

df.withColumn("bonus",my_fun(F.col("region"),F.col("amount")))\
  .withColumn("Bonus %",F.when(F.col("region").isin("North","South"),"10% bonus").otherwise("5% bonus")).show()


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


employees = [
    {"emp_id": 1, "name": "Arun",  "dept": "IT",       "experience": 2,  "salary": 45000},
    {"emp_id": 2, "name": "Meena", "dept": "HR",       "experience": 6,  "salary": 65000},
    {"emp_id": 3, "name": "Kiran", "dept": "IT",       "experience": 4,  "salary": 55000},
    {"emp_id": 4, "name": "Sachin","dept": "Finance",  "experience": 8,  "salary": 82000},
    {"emp_id": 5, "name": "Priya", "dept": "Finance",  "experience": 1,  "salary": 40000},
    {"emp_id": 6, "name": "Deepa", "dept": "HR",       "experience": 10, "salary": 90000},
    {"emp_id": 7, "name": "John",  "dept": "Marketing","experience": 3,  "salary": 48000},
    {"emp_id": 8, "name": "Riya",  "dept": "Marketing","experience": 7,  "salary": 70000}
]

spark  = SparkSession.builder.appName("udf").getOrCreate()

df = spark.createDataFrame(employees)      


In [ ]:
#without udf
df.withColumn(
    "experience_level",
    F.when((F.col("experience") > 0) & (F.col("experience") <= 3), "Junior")
     .when((F.col("experience") > 3) & (F.col("experience") <= 7), "Mid")
     .otherwise("Senior")).show()

In [ ]:
#udf
def exp(x):
    if x>0 and x<=3:
        return "Junior"
    elif x>3 and x<=7:
        return "Mid"
    else:
        return "Senior"

my_fun = F.udf(exp)

#with udf
df.withColumn("experience_level",my_fun(F.col("experience"))).show()  